# Question 1

In [1]:
""" DOWNLOADING TEXTBOOK FROM PROJECT GUTENBERG """

import requests
response = requests.get("http://www.gutenberg.org/files/11/11-0.txt")
text_data = response.text
# print(text_data)

# Question 2

In [16]:
""" PARSING THE DATA USING SENTENCE TOKENIZER """

from nltk.tokenize import sent_tokenize
sentences_list = sent_tokenize(text_data)
# print("\n".join(sentences_list))



""" REMOVING LINE-BREAKS, CARRIAGE-RETURNS AND NON-ALPHABETICAL CHARACTERS """
import re
def remove(string):
    string = string.replace('\n', ' ')\
                   .replace('\r', '')
    regex = re.compile('[^a-zA-Z0-9\s]')
    string = regex.sub('', string)
    return string
sentences_list = list(map(remove, sentences_list))


""" SPLITTING DATA INTO TRAINING AND TEST SET """
import random
random.shuffle(sentences_list)
sentences_list_train = sentences_list[:int(0.8 * len(sentences_list))]
sentences_list_test = sentences_list[int(0.8 * len(sentences_list)):]


""" CREATING VOCBULARY DICTIONARY """
countTotal = 0
dictVocab = dict()
for sentence in sentences_list_train:    
    for word in sentence.split():
        countTotal += 1
        if(word not in dictVocab):
            dictVocab[word] = 0
        dictVocab[word] += 1
        
        
        
""" ADDING <s> and </s> to the start and end of each sentence [TRAINING SET] """
for s_idx in range(len(sentences_list_train)):
    sentences_list_train[s_idx] = "<s> " + sentences_list_train[s_idx] + " </s>"    
    
""" ADDING <s> and </s> to the start and end of each sentence [TEST SET] """
for s_idx in range(len(sentences_list_test)):
    sentences_list_test[s_idx] = "<s> " + sentences_list_test[s_idx] + " </s>"
    
    
""" OUTPUT """
print("Some sentences from the training set - \n")
for s in sentences_list_train[:3]:
    print(s.split())
    print("\n")

Some sentences from the training set - 

['<s>', 'And', 'now', 'which', 'is', 'which', 'she', 'said', 'to', 'herself', 'and', 'nibbled', 'a', 'little', 'of', 'the', 'righthand', 'bit', 'to', 'try', 'the', 'effect', 'the', 'next', 'moment', 'she', 'felt', 'a', 'violent', 'blow', 'underneath', 'her', 'chin', 'it', 'had', 'struck', 'her', 'foot', '</s>']


['<s>', 'And', 'just', 'as', 'Id', 'taken', 'the', 'highest', 'tree', 'in', 'the', 'wood', 'continued', 'the', 'Pigeon', 'raising', 'its', 'voice', 'to', 'a', 'shriek', 'and', 'just', 'as', 'I', 'was', 'thinking', 'I', 'should', 'be', 'free', 'of', 'them', 'at', 'last', 'they', 'must', 'needs', 'come', 'wriggling', 'down', 'from', 'the', 'sky', '</s>']


['<s>', 'There', 'was', 'nothing', 'so', 'VERY', 'remarkable', 'in', 'that', 'nor', 'did', 'Alice', 'think', 'it', 'so', 'VERY', 'much', 'out', 'of', 'the', 'way', 'to', 'hear', 'the', 'Rabbit', 'say', 'to', 'itself', 'Oh', 'dear', '</s>']




# Question 3

In [23]:
from nltk import ngrams
    
def gen_ngrams(n):
    ngrams_list = []
    for sentence in sentences_list_train:
        ngrams_list = ngrams_list + list(ngrams(sentence.split(" "), n))
    return ngrams_list

unigrams  = gen_ngrams(1)
bigrams   = gen_ngrams(2)
trigrams  = gen_ngrams(3)
quadgrams = gen_ngrams(4)


""" MLE for unigrams """
unigramCount = dict()
unigramMLE = dict()
for key in unigrams:
    if(key not in unigramCount):
        unigramCount[key] = 0
    unigramCount[key] += 1
for key in unigramCount:
    unigramMLE[key] = unigramCount[key]/len(unigrams)

    
    
""" MLE for bigrams """
bigramCount = dict()
bigramMLE = dict()
for key in bigrams:
    if(key not in bigramCount):
        bigramCount[key] = 0
    bigramCount[key] += 1
for key in bigramCount:
    bigramMLE[key] = bigramCount[key]/unigramCount[(key[0], )]

    
    
""" MLE for trigrams """
trigramCount = dict()
trigramMLE = dict()
for key in trigrams:
    if(key not in trigramCount):
        trigramCount[key] = 0
    trigramCount[key] += 1
for key in trigramCount:
    trigramMLE[key] = trigramCount[key]/bigramCount[(key[0], key[1])]



""" MLE for quadgrams """
quadgramCount = dict()
quadgramMLE = dict()
for key in quadgrams:
    if(key not in quadgramCount):
        quadgramCount[key] = 0
    quadgramCount[key] += 1
for key in quadgramCount:
    quadgramMLE[key] = quadgramCount[key]/trigramCount[(key[0], key[1], key[2])]
    
    
print("MLE estimation of some bigrams - \n")
for key in list(bigramMLE.keys())[:3]:
    print(key, bigramMLE[key])
    print("\n")
    
    

MLE estimation of some bigrams - 

('<s>', 'And') 0.02631578947368421


('And', 'now') 0.017857142857142856


('now', 'which') 0.029411764705882353




# Question - 4

In [170]:
import math
import numpy
class System:
    def generate_sentence(self, model_name):
        name2mle = {"unigram": unigramMLE, "bigram": bigramMLE, "trigram": trigramMLE, "quadgram": quadgramMLE}
        MLE = name2mle[model_name]
        sentence = []
    
        # GENERATE FIRST RANDOM SENTENCE
        start = "<s>"
        key_list = []
        value_list = []
        for key in MLE:
            if(key[0] == start):
                key_list.append(key)
                value_list.append(MLE[key])
                
        # NORMALIZE
        sum_value_list = sum(value_list)
        for zz in range(len(value_list)):
            value_list[zz]/=sum_value_list
            
        randomResult = numpy.random.multinomial(100, value_list, 1).tolist()
        sampleIndex = randomResult[0].index(max(randomResult[0]))
        sentence += list(key_list[sampleIndex])
        
        iterations = 0
        while(sentence[-1] != "</s>"):
            start = sentence[-1]
            key_list = []
            value_list = []
            for key in MLE:
                if(key[0] == start):
                    key_list.append(key)
                    value_list.append(MLE[key])
            
            sum_value_list = sum(value_list)
            for zz in range(len(value_list)):
                value_list[zz]/=sum_value_list
            
            randomResult = numpy.random.multinomial(100, value_list, 1).tolist()
            sampleIndex = randomResult[0].index(max(randomResult[0]))
            sentence += list(key_list[sampleIndex])[1:]
            iterations += 1
            if(iterations > 10):
                break
        
        return sentence
                
    def get_probability(self, sentence, model_name):
        num2mle  = {1: unigramMLE, 2: bigramMLE, 3: trigramMLE, 4: quadgramMLE}        
        name2num = {"unigram": 0, "bigram": 1, "trigram": 2, "quadgram": 3}
        N = name2num[model_name]
        logProb = 1.0
        sentence_as_list = sentence.split()
        
        for idx in range(1, len(sentence_as_list)):
            curr_tup = []
            for j in range(N, -1, -1):
                if(idx-j >= 0):
                    curr_tup.append(sentence_as_list[idx-j])
            curr_tup = tuple(curr_tup) 
            tempProb = num2mle[len(curr_tup)][curr_tup]
            logProb += math.log2(tempProb)
        
        return logProb
            
system = System()
model_name_1 = "quadgram"
print("Sample sentence generated using %s model - " % model_name_1)
print(system.generate_sentence(model_name_1))
print("\n")
model_name_2 = "bigram"
print('Probability [In log space] of the sentence - "I must have been changed for Mabel"')
print(system.get_probability("<s> I must have been changed for Mabel </s>", model_name_2))

Sample sentence generated using quadgram model - 
['<s>', 'Very', 'much', 'indeed', 'and', 'much', 'sooner', 'than', 'she', 'had', 'said', 'that', 'day', 'about', 'it', 'and', 'to', 'hear', 'his', 'shrill', 'little', 'voice', 'alongCatch', 'him', 'you', 'charge', 'for', 'the', 'Rabbits', 'little', 'white', 'one', 'in', 'by', 'way', 'of', 'keeping']


Probability [In log space] of the sentence - "I must have been changed for Mabel"
-27.303160370643322


# Question 5

In [140]:
""" ADD-1 SMOOTHING """

vocabSize = len(unigrams)
bigramAddOneMLE = dict()
for key in bigramCount:
    bigramAddOneMLE[key] = (bigramCount[key] + 1)/(unigramCount[(key[0], )] + vocabSize)
    
for key in list(bigramMLE.keys())[:3]:
    print("-----------------------------------------")
    print(key)
    print("MLE before Add-1 smoothing - %f" % bigramMLE[key])
    print("After Add-1 smoothing - %f" % bigramAddOneMLE[key])
    print("-----------------------------------------")


-----------------------------------------
('<s>', 'And')
MLE before Add-1 smoothing - 0.026316
After Add-1 smoothing - 0.000903
-----------------------------------------
-----------------------------------------
('And', 'now')
MLE before Add-1 smoothing - 0.017857
After Add-1 smoothing - 0.000078
-----------------------------------------
-----------------------------------------
('now', 'which')
MLE before Add-1 smoothing - 0.029412
After Add-1 smoothing - 0.000078
-----------------------------------------
False


#### A drastic change in count occurs for the bigram - ('<s\>', 'And'). This reduced count will be used to estimate the count of bigrams that we have never seen.

# Question 6

In [106]:
""" GOOD TURING SMOOTHING - FINDING VALUE OF D"""

# CREATE FREQUENCY OF FREQUENCIES
freqDict = dict()
for k in bigramCount.values():
    if(k not in freqDict):
        freqDict[k] = 0
    freqDict[k] += 1

freqDict[0] = len(bigrams)
newCount = {}
for k in range(10, -1, -1):
    newCount[k] = ((k + 1) * freqDict[k + 1]) / freqDict[k]

for k in list(newCount.keys())[::-1]:
    print(k, newCount[k], k - newCount[k])

print("VALUE OF D OBTAINED IS = %f" % ((1-newCount[1]+2-newCount[2]+3-newCount[3]+4-newCount[4]+5-newCount[5])/5))

0 0.448150833937636 -0.448150833937636
1 0.2948579647608774 0.7051420352391227
2 1.0426829268292683 0.9573170731707317
3 1.7894736842105263 1.2105263157894737
4 3.392156862745098 0.607843137254902
5 3.745664739884393 1.254335260115607
6 4.277777777777778 1.7222222222222223
7 7.393939393939394 -0.3939393939393936
8 6.049180327868853 1.9508196721311473
9 7.804878048780488 1.1951219512195124
10 8.25 1.75
VALUE OF D OBTAINED IS = 0.947033


In [126]:
""" GOOD TURING SMOOTHING - FINDING MLEs """

bigramGoodTuringMLE = {}
for key in bigramCount:
    if(bigramCount[key] in newCount):
        bigramGoodTuringMLE[key] = newCount[bigramCount[key]]/unigramCount[(key[0], )]
    else:
        bigramGoodTuringMLE[key] = bigramMLE[key]
        
for key in list(bigramMLE.keys())[1:4]:
    print("-----------------------------------------")
    print(key)
    print("MLE before Good Turing smoothing - %f" % bigramMLE[key])
    print("After Good Turing smoothing - %f" % bigramGoodTuringMLE[key])
    print("-----------------------------------------")

-----------------------------------------
('And', 'now')
MLE before Good Turing smoothing - 0.017857
After Good Turing smoothing - 0.005265
-----------------------------------------
-----------------------------------------
('now', 'which')
MLE before Good Turing smoothing - 0.029412
After Good Turing smoothing - 0.008672
-----------------------------------------
-----------------------------------------
('which', 'is')
MLE before Good Turing smoothing - 0.030303
After Good Turing smoothing - 0.008935
-----------------------------------------


# Question 7

In [154]:
""" PERPLEXITY VALUE FOR TEST DATASET USING ADD-1 SMOOTHING """

def gen_bigrams_test():
    ngrams_list = []
    for sentence in sentences_list_test:
        ngrams_list = ngrams_list + list(ngrams(sentence.split(" "), 2))
    return ngrams_list
bigramsTest = gen_bigrams_test()

testSetSize = 0
for sentence in sentences_list_test:
    testSetSize += len(sentence.split(" "))

perplexityAddOne = 1.0
for k in bigramsTest:
    if(k in bigramAddOneMLE):
        perplexityAddOne *= ((1.0/bigramAddOneMLE[k]) ** (1.0/testSetSize))
    else:
        perplexityAddOne *= (len(unigrams) ** (1.0/testSetSize))

print("Perplexity value for Add-1 smoothing - ")
print(perplexityAddOne)

Perplexity value for Add-1 smoothing - 
5819.094621780029


In [155]:
""" PERPLEXITY VALUE FOR TEST DATASET USING GOOD TURING SMOOTHING """

perplexityGoodTuring = 1.0
for k in bigramsTest:
    if(k in bigramGoodTuringMLE):
        perplexityGoodTuring *= ((1.0/bigramGoodTuringMLE[k]) ** (1.0/testSetSize))
    else:
        perplexityGoodTuring *= ((len(unigrams)/newCount[0]) ** (1.0/testSetSize))

print("Perplexity value for Good Turing smoothing - ")
print(perplexityGoodTuring)

Perplexity value for Good Turing smoothing - 
719.0790952688778


#### Since, perplexity value for Good Turing smoothing is less than Add-1 smoothing, we can say that Good Turing smoothing performs better on the test set as compared to Add-1 smoothing.